# Geocoding Police Fire Departments and Mapping to CSI360 Customer Locations

- Import and data from CSI360 Police and Fire List.
- Geocode addresses to obtain latitude and longitude coordinates.
- Insert latitude and longitude columns into data file.
- Check each department location and identify if it is within 100 miles of the customer locations.
- Separate the dataframe for departments within 100 miles for each customer location.
- Export the data to CSV files for further analysis and reporting.

In [1]:
# Dependencies and Setup
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic
# import folium # Use to create html map
# from folium import plugins # Use to create html map
from ratelimit import limits, sleep_and_retry
import time
import numpy as np # Use for Haversine formula

# Manually load API key from .env file. # Remove code associated with dotenv if this works
with open('C:/Users/jchan/csi360_fire_police/lefd-custy-targets/.env', 'r') as f:
    for line in f:
        if line.startswith('GOOGLE_MAPS_API_KEY'):
            google_api_key = line.split('=')[1].strip()

In [2]:
# Store filepath in a variable
lefd_1_data = 'resources/data/lefd_1_data.csv'
csi360_customers = 'resources/data/csi360_customers.csv'

# Read each of the respective files (police, fire, agency_n, agency_addrs) and store into Pandas dataframe
lefd_1_data_df = pd.read_csv(lefd_1_data)
csi360_customers_df = pd.read_csv(csi360_customers)

# Convert the 'zip' column to integer, then back to string to remove decimals
csi360_customers_df['zip'] = csi360_customers_df['zip'].fillna("").apply(lambda x: str(int(float(x))) if x != "" else "")

display(lefd_1_data_df.head(), csi360_customers_df.head())

,agency_name,agency_type,sworn_active_persnl,hq_addr1,hq_addr2,hq_city,hq_state,hq_zip,addr1,addr2,po_box,city,state,county,zip,hq_ph,hq_fax,org_type,website,fips
0,New York City Police Department,Local police department,36023,,,,,,"One Police Plaza, Room 1400",,,New York,NY,New York,10038,,,,,36061
1,Chicago Police Dept,Local police department,13354,,,,,,3510 S Michigan,,,Chicago,IL,Cook,60653,,,,,17031
2,Los Angeles Police Department,Local police department,9727,,,,,,150 N. Los Angeles Street,,,Los Angeles,CA,Los Angeles,90012,,,,,6037
3,Los Angeles County Sheriff's Office,Sheriff's office,9461,,,,,,Unit 1,4700 W. Ramona Blvd.,,Monterey Park,CA,Los Angeles,91754,,,,,6037
4,California Highway Patrol,Primary state law enforcement agency,7202,,,,,,2555 First Ave,,,Sacramento,CA,Sacramento,95818,,,,,6067


,company,contact,State,Last Contact,Status,zip
0,Berkeley County Sheriff's Office,Lt. Geno Alterio,SC,1/17/2023,Custy-5/20,29461
1,Blytheville Police Department,Vanessa Stewart,AZ,10/27/2023,Custy-10/20,72315
2,FEMA - Idaho,Keith Richey,NaN,NaN,NaN,
3,Franklin County Sheriff's Office,Rhonda Coyne,NC,8/22/2024,Custy-3/24,27549
4,Grant Parish Sheriff's Office,Cade Fletcher,LA,2/1/2023,Custy-8/22,71417


In [3]:
## Geocode CSI360 customer zip codes

In [4]:
# Initialize geolocator. 
geolocator = GoogleV3(api_key=google_api_key)

In [5]:
# Function to get geocode based on zip code

def get_lat_long(zip_code):
    if zip_code == "":  # Skip empty ZIP codes
        return None, None
    try:
        # Use the ZIP code directly to get the location
        location = geolocator.geocode(zip_code)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for ZIP code: {zip_code}")  # Log failed geocodes
            return None, None
    except Exception as e:
        print(f"Error geocoding {zip_code}: {e}")
        return None, None

# Apply the geocode function to get Latitude and Longitude for each row in the cleaned DataFrame
csi360_customers_df['Latitude'], csi360_customers_df['Longitude'] = zip(*csi360_customers_df['zip'].apply(get_lat_long))

# Fill NaN values in 'Latitude' and 'Longitude' with empty strings
csi360_customers_df['Latitude'] = csi360_customers_df['Latitude'].fillna("")
csi360_customers_df['Longitude'] = csi360_customers_df['Longitude'].fillna("")

# Print the first few rows to verify
print(csi360_customers_df.head())


                            company           contact State Last Contact  \
0  Berkeley County Sheriff's Office  Lt. Geno Alterio    SC    1/17/2023   
1     Blytheville Police Department   Vanessa Stewart    AZ   10/27/2023   
2                      FEMA - Idaho      Keith Richey   NaN          NaN   
3  Franklin County Sheriff's Office      Rhonda Coyne    NC    8/22/2024   
4     Grant Parish Sheriff's Office     Cade Fletcher    LA     2/1/2023   

        Status    zip   Latitude  Longitude  
0   Custy-5/20  29461  33.146521 -79.986407  
1  Custy-10/20  72315  35.893377 -89.906631  
2          NaN                               
3   Custy-3/24  27549  36.073028 -78.247615  
4   Custy-8/22  71417  31.532732 -92.638779  


## Geocode Colorado Fire Deptartment Addresses


In [6]:
# Function to get latitude and longitude from address
# Set rate limit to 25 requests per second
# Google geocoding API allows around 50 queries per second, I've adjusted to 25 to be safe
RATE_LIMIT = 25  # requests per second
TIME_PERIOD = 1  # time period in seconds

# Function to get latitude and longitude from address with rate limiter
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_PERIOD)
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

In [9]:
# Combine address fields and get latitude/longitude
lefd_1_data_df['Full_Address'] = lefd_1_data_df['addr1'] + ', ' + lefd_1_data_df['city'] + ', ' + lefd_1_data_df['state'] + ' ' + lefd_1_data_df['zip'].astype(str)
lefd_1_data_df['Latitude'], lefd_1_data_df['Longitude'] = zip(*lefd_1_data_df['Full_Address'].apply(get_lat_long))
print(lefd_1_data_df.head())

                           agency_name                           agency_type  \
0      New York City Police Department               Local police department   
1                  Chicago Police Dept               Local police department   
2        Los Angeles Police Department               Local police department   
3  Los Angeles County Sheriff's Office                      Sheriff's office   
4            California Highway Patrol  Primary state law enforcement agency   

   sworn_active_persnl hq_addr1 hq_addr2 hq_city hq_state hq_zip  \
0                36023                                             
1                13354                                             
2                 9727                                             
3                 9461                                             
4                 7202                                             

                         addr1                 addr2  ...       county    zip  \
0  One Police Plaza, Room 140

In [11]:
# Save the dataframes to a new CSV file
csi360_customers_df.to_csv('C:/Users/jchan/csi360_fire_police/lefd-custy-targets/resources/Output/csi360_custy_latlong.csv', index=False)
lefd_1_data_df.to_csv('C:/Users/jchan/csi360_fire_police/lefd-custy-targets/resources/Output/lefd_latlong.csv', index=False)

## Use Haversine Formula to calculate distances between latitude and longitude pairs 
### Source: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas

In [12]:
# Load the CSV files into dataframes
csi360_customers_df = pd.read_csv('resources/output/csi360_custy_latlong.csv')
lefd_latlong_df = pd.read_csv('resources/output/lefd_latlong.csv')

print(csi360_customers_df.head(2), lefd_latlong_df.head(2))

                            company           contact State Last Contact  \
0  Berkeley County Sheriff's Office  Lt. Geno Alterio    SC    1/17/2023   
1     Blytheville Police Department   Vanessa Stewart    AZ   10/27/2023   

        Status      zip   Latitude  Longitude  
0   Custy-5/20  29461.0  33.146521 -79.986407  
1  Custy-10/20  72315.0  35.893377 -89.906631                          agency_name              agency_type  \
0  New York City Police Department  Local police department   
1              Chicago Police Dept  Local police department   

   sworn_active_persnl hq_addr1 hq_addr2 hq_city hq_state hq_zip  \
0                36023                                             
1                13354                                             

                         addr1 addr2  ...    county    zip hq_ph hq_fax  \
0  One Police Plaza, Room 1400        ...  New York  10038                
1              3510 S Michigan        ...      Cook  60653                

   org

In [13]:
# Haversine formula to calculate the distance between two lat/lon pairs
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # Earth radius in miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Function to filter agencies within 100 miles using Haversine formula
def filter_agencies_within_100_miles(lefd_latlong_df, csi360_customers_df):
    results = []

 # Iterate over each company in csi360_customers_df
    for _, company_row in csi360_customers_df.iterrows():
        company_name = company_row['company']
        company_coords = (company_row['Latitude'], company_row['Longitude'])

 # Check each agency in lefd_latlong_df
        for _, agency_row in lefd_latlong_df.iterrows():
            try:
                agency_coords = (agency_row['Latitude'], agency_row['Longitude'])
                distance = haversine(company_coords[0], company_coords[1], agency_coords[0], agency_coords[1])
                
                # If distance is within 100 miles, store the result
                if distance <= 100:
                    results.append({
                        'company': company_name,
                        **agency_row.to_dict()  # Include all columns from lefd_latlong_df
                    })
            except Exception as e:
                print(f"Error calculating distance for {agency_row['agency_name']}: {e}")
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Print the first 5 results for verification
    print(results_df.head(5))

    # Save the results to a CSV file
    results_df.to_csv('C:/Users/jchan/csi360_fire_police/lefd-custy-targets/resources/output/lefd_100mi_csicusty.csv', index=False)
  
    return results_df

# Use the function with the loaded data
filter_agencies_within_100_miles(lefd_latlong_df, csi360_customers_df)


                            company  \
0  Berkeley County Sheriff's Office   
1  Berkeley County Sheriff's Office   
2  Berkeley County Sheriff's Office   
3  Berkeley County Sheriff's Office   
4  Berkeley County Sheriff's Office   

                                       agency_name  \
0                    South Carolina Highway Patrol   
1  Savannah-Chatham Metropolitan Police Department   
2                 Richland County Sheriff's Office   
3                     Charleston Police Department   
4                       Columbia Police Department   

                            agency_type  sworn_active_persnl hq_addr1  \
0  Primary state law enforcement agency                  967            
1               Local police department                  534            
2                      Sheriff's office                  512            
3               Local police department                  382            
4               Local police department                  351            

 

,company,agency_name,agency_type,sworn_active_persnl,hq_addr1,hq_addr2,hq_city,hq_state,hq_zip,addr1,...,county,zip,hq_ph,hq_fax,org_type,website,fips,Full_Address,Latitude,Longitude
0,Berkeley County Sheriff's Office,South Carolina Highway Patrol,Primary state law enforcement agency,967,,,,,,10311 Wilson Blvd,...,Richland,29016,,,,,45079,"10311 Wilson Blvd, Blythewood, SC 29016",34.178294,-80.973790
1,Berkeley County Sheriff's Office,Savannah-Chatham Metropolitan Police Department,Local police department,534,,,,,,Post Office Box 8032,...,Chatham,31412,,,,,13051,"Post Office Box 8032, Savannah, GA 31412",32.079528,-81.090081
2,Berkeley County Sheriff's Office,Richland County Sheriff's Office,Sheriff's office,512,,,,,,211 W Market St,...,Richland,29223,,,,,45079,"211 W Market St, Columbia, SC 29223",34.105838,-80.920774
3,Berkeley County Sheriff's Office,Charleston Police Department,Local police department,382,,,,,,Post Office Box 420,...,Charleston,29403,,,,,45019,"Post Office Box 420, Charleston, SC 29403",32.800724,-79.950048
4,Berkeley County Sheriff's Office,Columbia Police Department,Local police department,351,,,,,,Po Box 10,...,Richland,29201,,,,,45079,"Po Box 10, Columbia, SC 29201",33.987339,-81.036821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Yoakum Police Department,Travis County Sheriff's Office,Sheriff's office,290,,,,,,P.O. Box 1748,...,Travis,78767,,,,,48453,"P.O. Box 1748, Austin, TX 78767",30.269417,-97.739694
224,Yoakum Police Department,Texas Alcoholic Beverage Commission,Special jurisdiction,277,,,,,,5806 Mesa Drive,...,Travis,78731,,,,,48453,"5806 Mesa Drive, Austin, TX 78731",30.342110,-97.771212
225,Yoakum Police Department,Round Rock Police Department,Local police department,133,,,,,,2701 N. Mays St.,...,Williamson,78665,,,,,48491,"2701 N. Mays St., Round Rock, TX 78665",30.540826,-97.687031
226,Yoakum Police Department,Sugar Land Police Department,Local police department,130,,,,,,PO BOX 110,...,Fort Bend,77478,,,,,48157,"PO BOX 110, Sugar Land, TX 77478",29.618521,-95.609001
